### Load the book

In [85]:
with open("miracle_in_the_andes.txt", "r", encoding="utf8") as file:
    book = file.read()

### How many chapters are there in the book?

##### With string method

In [2]:
book.count("Chapter") # not accurate, because it catches cases where the word in used in the text, too

11

##### With regex

In [3]:
import re

In [4]:
pattern = re.compile("Chapter [0-9]")

In [5]:
re.findall(pattern, book)

['Chapter 1',
 'Chapter 2',
 'Chapter 3',
 'Chapter 4',
 'Chapter 5',
 'Chapter 6',
 'Chapter 7',
 'Chapter 8',
 'Chapter 9',
 'Chapter 1']

In [6]:
pattern = re.compile("Chapter [0-9]+")

In [7]:
findings = re.findall(pattern, book)

In [8]:
findings

['Chapter 1',
 'Chapter 2',
 'Chapter 3',
 'Chapter 4',
 'Chapter 5',
 'Chapter 6',
 'Chapter 7',
 'Chapter 8',
 'Chapter 9',
 'Chapter 10']

In [9]:
len(findings)

10

### Which are the sentences where "love" is used?

In [10]:
# [^.] means everythong except the period sign.
# [A-Z]{1} pattern should have exactly one capital letter
# * is 0 or more times
# + is 1 or more times
# {1} is exactly one time
pattern = re.compile("[A-Z]{1}[^.]*[^a-zA-Z]love[^a-zA-Z][^.]*.") 
findings = re.findall(pattern, book)
len(findings)

67

### What are the most used words?

In [11]:
pattern = re.compile("[a-zA-Z]+")
findings = re.findall(pattern, book.lower()) # using lower() to avoid counting capitalized strings as different words
len(findings)

86798

In [12]:
# Below code creates a dict where keys are words, and values represent the count of how many times each word has appeared

dictionary = {}
for word in findings:
    if word in dictionary.keys():
        dictionary[word] = dictionary[word] + 1
    else:
        dictionary[word] = 1

In [13]:
# Create list with the values in the first position, followed by the keyword

dictionary_list = [(value, key) for (key, value) in dictionary.items()]

# We sort the list and put the most occurences on top

sorted(dictionary_list, reverse=True)[:10] # [:10] shows only first 10 items from the list

[(5346, 'the'),
 (2795, 'and'),
 (2729, 'i'),
 (2400, 'to'),
 (2060, 'of'),
 (1566, 'a'),
 (1430, 'was'),
 (1419, 'in'),
 (1226, 'we'),
 (1169, 'my')]

### Extract paragraphs where love is used

In [28]:
# Regex pattern using lookbehind and lookahead:
# (?<=\n\n)        - Positive lookbehind: Asserts that the match is preceded by two newline characters,
#                    indicating the start of a new paragraph.
# (.*?\blove\b.*?)  - Captures any text that contains the word "love" as a whole word, 
#                    allowing for any characters (including newlines) before and after "love".
# (?=\n\n)         - Positive lookahead: Asserts that the match is followed by two newline characters,
#                    indicating the end of a paragraph.
pattern = re.compile(r"(?<=\n\n)(.*?\blove\b.*?)(?=\n\n)")
findings = re.findall(pattern, book)
len(findings)
#findings

40

### Extract chapter titles

In [35]:
# The dot (.) matches any single character except newline characters.
# The plus sign (+) is a quantifier in regex that indicates one or more occurrences of the preceding element.

pattern = re.compile(r"Chapter [0-9]+\n\n(.+)")
findings = re.findall(pattern, book)
findings

['Before',
 'Everything Precious',
 'A Promise',
 'Breathe Once More',
 'Abandoned',
 'Tomb',
 'East',
 'The Opposite of Death',
 'I See a Man',
 'After']

### Function that finds the occurence of any word

In [49]:
def finder(word):
    pattern = re.compile(r"\b" + word + r"\b")
    findings = re.findall(pattern, book.lower())
    if findings:
        print (len(findings))
    else:
        print(f'The book doesn\'t contain the word "{word}"')

# looks like we can use an fstring + rstring combo as well:
# pattern = re.compile(fr'\b{word}\b')

### Call the function

In [47]:
finder("love")

83


In [48]:
finder("hate")

The book doesn't contain the word "hate"


### The most used words (non-articles)

In [68]:
sorted_dlist = sorted(dictionary_list, reverse=True) # defined above, contains articles

In [55]:
!pip3.11 install nltk

In [62]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dobri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
english_stopwords[:5]

['i', 'me', 'my', 'myself', 'we']

In [69]:
filtered_words = []
for count,word in sorted_dlist:
    if word not in english_stopwords:
        filtered_words.append((word, count))

In [71]:
filtered_words[:10]

[('would', 575),
 ('us', 519),
 ('said', 292),
 ('roberto', 284),
 ('could', 252),
 ('one', 249),
 ('snow', 227),
 ('mountain', 183),
 ('time', 182),
 ('like', 165)]

### Sentiment Analysis: Which is the most positive and the most negative chapter?

#### An Example:

In [73]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [81]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dobri\AppData\Roaming\nltk_data...


True

In [82]:
analyzer = SentimentIntensityAnalyzer()

In [77]:
analyzer

nltk.sentiment.vader.SentimentIntensityAnalyzer

In [83]:
analyzer.polarity_scores("Look how beautiful the trees are. I love them.")

{'neg': 0.0, 'neu': 0.426, 'pos': 0.574, 'compound': 0.8442}

In [84]:
analyzer.polarity_scores(book)

{'neg': 0.116, 'neu': 0.76, 'pos': 0.125, 'compound': 1.0}

#### Chapters sentiment analysis

In [86]:
pattern = re.compile("Chapter [0-9]+")
chapters = re.split(pattern, book)
chapters = chapters[1:] # gets rid of an empty string at the beginning of the list

In [88]:
for index,chapter in enumerate(chapters):
    scores = analyzer.polarity_scores(chapter)
    print(f"Chapter {index + 1}: {scores}")

Chapter 1: {'neg': 0.061, 'neu': 0.779, 'pos': 0.16, 'compound': 1.0}
Chapter 2: {'neg': 0.12, 'neu': 0.726, 'pos': 0.154, 'compound': 0.9991}
Chapter 3: {'neg': 0.145, 'neu': 0.751, 'pos': 0.105, 'compound': -0.9999}
Chapter 4: {'neg': 0.141, 'neu': 0.721, 'pos': 0.138, 'compound': -0.9963}
Chapter 5: {'neg': 0.118, 'neu': 0.742, 'pos': 0.141, 'compound': 0.9997}
Chapter 6: {'neg': 0.124, 'neu': 0.761, 'pos': 0.115, 'compound': -0.9979}
Chapter 7: {'neg': 0.136, 'neu': 0.761, 'pos': 0.103, 'compound': -0.9999}
Chapter 8: {'neg': 0.12, 'neu': 0.786, 'pos': 0.094, 'compound': -0.9998}
Chapter 9: {'neg': 0.097, 'neu': 0.824, 'pos': 0.079, 'compound': -0.9996}
Chapter 10: {'neg': 0.086, 'neu': 0.733, 'pos': 0.181, 'compound': 1.0}
